In [6]:
import pandas as pd
from pathlib import Path

## Step 1: Remove Useless columns
In this step, we will load all **Shopify** orders and then select only valuable columns for our models.
belong to clean.py

In [7]:

raw_orders_df = pd.read_csv('orders_export_1.csv')
orders_selected_columns = raw_orders_df[["Email", # identifiers
                                        "Total", # order total
                                        "Discount Amount", 
                                        "Created at", # order creation date
                                        "Lineitem quantity",
                                        "Lineitem price"]]
orders_selected_columns.to_csv("step1.csv")

print(orders_selected_columns)

                            Email   Total  Discount Amount  \
0                lvaane@gmail.com   81.48            20.37   
1     Annekedooijewaard@gmail.com   70.06             7.79   
2            hankebertram@msn.com   80.68            20.17   
3            hankebertram@msn.com     NaN              NaN   
4               svelmbt@gmail.com   47.61             5.29   
...                           ...     ...              ...   
5825           r.hurk@hotmail.com  246.99            43.56   
5826           r.hurk@hotmail.com     NaN              NaN   
5827           r.hurk@hotmail.com     NaN              NaN   
5828     charlotte.roke@gmail.com   23.35             2.60   
5829            kristy@pousset.be   43.11             4.79   

                     Created at  Lineitem quantity  Lineitem price  
0     2025-05-31 06:53:36 +0200                  3           33.95  
1     2025-05-31 06:44:57 +0200                  3           25.95  
2     2025-05-30 23:59:48 +0200                 

## Step 2: Normalize strings, dates, numeric values and drop rows where the email is missing
Columns with a missing email have an uknown customer, there is no value in predicting the NPD for unknown customers
belong to clean.py

In [8]:
# Normalize columns
orders_selected_columns["Created at"] = pd.to_datetime(orders_selected_columns["Created at"],utc=True)

# We have some columns "Total", "Discount Amount", and "Lineitem quantity" that may currently be strings.So convert them into float

cols_to_float = ["Total", "Discount Amount", "Lineitem quantity"]
for col in cols_to_float:
    raw_orders_df[col] = pd.to_numeric(raw_orders_df[col])

# make all the emails lowercase
orders_selected_columns["Email"] = orders_selected_columns["Email"].str.lower().str.strip()
df = orders_selected_columns[orders_selected_columns["Email"].notna() & (orders_selected_columns["Email"] != "")]

# After subsetting, the index may be non‐consecutive (e.g., 0, 2, 5, 7). 
# Resetting the index gives us a clean 0..N−1 index.
df.reset_index(drop=True, inplace=True)
df["Created at"] = pd.to_datetime(df['Created at'].dt.date,format="yyyy-mm-dd")

df.to_csv("step2.csv")
print(df)


                            Email   Total  Discount Amount Created at  \
0                lvaane@gmail.com   81.48            20.37 2025-05-31   
1     annekedooijewaard@gmail.com   70.06             7.79 2025-05-31   
2            hankebertram@msn.com   80.68            20.17 2025-05-30   
3            hankebertram@msn.com     NaN              NaN 2025-05-30   
4               svelmbt@gmail.com   47.61             5.29 2025-05-30   
...                           ...     ...              ...        ...   
5825           r.hurk@hotmail.com  246.99            43.56 2025-03-01   
5826           r.hurk@hotmail.com     NaN              NaN 2025-03-01   
5827           r.hurk@hotmail.com     NaN              NaN 2025-03-01   
5828     charlotte.roke@gmail.com   23.35             2.60 2025-03-01   
5829            kristy@pousset.be   43.11             4.79 2025-03-01   

      Lineitem quantity  Lineitem price  
0                     3           33.95  
1                     3           25.95

C:\Users\mdtas\AppData\Local\Temp\ipykernel_24176\2478249081.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders_selected_columns["Created at"] = pd.to_datetime(orders_selected_columns["Created at"],utc=True)
C:\Users\mdtas\AppData\Local\Temp\ipykernel_24176\2478249081.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders_selected_columns["Email"] = orders_selected_columns["Email"].str.lower().str.strip()


## Step 3 : Feature engineering 
Here we are going to do feature engineering. At end of the feature engineering each row we will have the following columns to our dataset
- Days since first purchase date(this is the different between first purchase date - current date )
- Days since second last purchase date (this is the different between second last purchase date - current date)
- Purchase interval days between orders. Here will have few columns and they are 
    - Purchase interval between 1st and 2nd order
    - Purchase interval between 2nd and 3nd order
    - Purchase interval between 3st and 4nd order
    - so on so forth . It will depend on how on how many orders a customer have
- Average order value (total / nr of order of each customer)
- Target column in days(2nd last purchase - last purchase date)

**Note**
- Let's think about this brenda.bruinenberg@hotmail.com, this customer. She ordered 7 products on the same date, meaning it's one order with 7 items. So this customer never come back again . So we will remove those customer churn. 
- We are going to exclude the last order of each customer if he/she order 2 or more orders not 2 or more items  to avoid data leakage for predicting next purchase date, meaning 2 or more order on different dates.

belong to feature_engineer.py

In [9]:

STEP2_PATH = Path("step2.csv")        # cleaned orders (line‑item level)
# Use the current UTC timestamp at runtime
TODAY = pd.Timestamp.now(tz="UTC").normalize()
MAX_GAPS   = 10                                 # how many interval columns to keep

# 1. Load & tidy the raw orders
orders_raw = pd.read_csv(STEP2_PATH)

# Drop accidental index column if present
orders_raw = orders_raw.drop(columns=["Unnamed: 0"], errors="ignore")

# Normalise headers just in case
orders_raw.columns = orders_raw.columns.str.strip()

# Parse timestamp with timezone
orders_raw["Created at"] = pd.to_datetime(orders_raw["Created at"], utc=True)
orders_raw.to_csv("orders_raw.csv")

# Deduplicate so that one row == one order even if multi‑item
# 0   2025-04-10 00:00:00  (T-shirt)
# 1   2025-04-10 00:00:00  (Jeans)
# 2   2025-04-10 00:00:00  (Cap)
# 3   2025-05-15 00:00:00  (Shoes)

# 0   2025-04-10 00:00:00
# 1   2025-05-15 00:00:00


orders = (
    orders_raw.sort_values(["Email", "Created at"])
              .drop_duplicates(subset=["Email", "Created at"])
              .reset_index(drop=True)
)
orders.to_csv("orders.csv")

# 2. Build feature rows
rows = []
for email, grp in orders.groupby("Email", sort=False):
    dates = grp["Created at"].sort_values().reset_index(drop=True)
    
    # Keep only customers with ≥2 orders
    if len(dates) < 2:
        continue
    
    
    #     dates = [
    #     2025-01-10,   # first order
    #     2025-02-15,   # second order
    #     2025-04-02,   # third order (second-last)
    #     2025-05-20    # fourth order (last)
    # ]

    target_gap = (dates.iloc[-1] - dates.iloc[-2]).days
    
    #  drop last order to avoid data leakage
    dates_kept = dates.iloc[:-1]


    num_orders = len(dates)
    total_spend = (
        grp.loc[
            grp["Created at"].isin(dates),
            "Total"
        ]
        .astype(float)
        .sum()
    )
    
    #how much money the customer spent prior to their last purchase.
    total_spend       = grp.loc[grp["Created at"].isin(dates), "Total"].astype(float).sum()
    avg_order_value   = total_spend / num_orders
    days_since_first  = (TODAY - dates_kept.iloc[0]).days
    days_since_2ndlast= (TODAY - dates_kept.iloc[-1]).days
    
    # Consecutive gaps
    gaps = dates_kept.diff().dt.days.dropna().astype("Int64").tolist()
    gaps = (gaps + [pd.NA]*MAX_GAPS)[:MAX_GAPS]
    
    row = {
        "email": email,
        "num_orders": num_orders,
        "total_spend": total_spend,
        "avg_order_value": avg_order_value,
        "days_since_first_purchase": days_since_first,
        "days_since_second_last_purchase": days_since_2ndlast,
        "target_gap_days": target_gap,
    }
    row.update({f"interval_{i+1}_{i+2}": g for i, g in enumerate(gaps)})
    rows.append(row)

features_df = pd.DataFrame(rows)

# Fill missing intervals with -1
interval_cols = [c for c in features_df.columns if c.startswith("interval_")]
features_df[interval_cols] = features_df[interval_cols].fillna(-1)

# ---------------------------------------------------------------------
# 3. Save & preview
# ---------------------------------------------------------------------
features_df.to_csv("final_features.csv")




C:\Users\mdtas\AppData\Local\Temp\ipykernel_24176\3520240136.py:95: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  features_df[interval_cols] = features_df[interval_cols].fillna(-1)


belong to model 

In [10]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

# ------------------------------------------------------------------
# 1. File paths
# ------------------------------------------------------------------
FEATURES_PATH = Path("final_features.csv")   # one row per customer
ORDERS_PATH   = Path("step2.csv")            # one row per order
OUT_PRED_PATH = Path("predicted_next_purchase_dates_fixed.csv")

# ------------------------------------------------------------------
# 2. Load feature matrix (has target_gap_days and email)


feat = (
    pd.read_csv(FEATURES_PATH)
      .drop(columns=["Unnamed: 0"], errors="ignore")
)


# ------------------------------------------------------------------
# 3. Reconstruct each customer's *last* order date
# ------------------------------------------------------------------
orders = (
    pd.read_csv(ORDERS_PATH)
      .drop(columns=["Unnamed: 0"], errors="ignore")
)
orders["Created at"] = pd.to_datetime(orders["Created at"], utc=True)

# make sure one row per email
last_orders = (
    orders.groupby("Email")["Created at"]
          .last()
          .rename("last_order_date")
          .reset_index()
)

# ------------------------------------------------------------------
# 4. Merge last_order_date into the features
# ------------------------------------------------------------------
df = feat.merge(last_orders, left_on="email", right_on="Email", how="left")

# Sanity check
if df["last_order_date"].isna().any():
    raise ValueError("Some emails missing last_order_date after merge.")

# ------------------------------------------------------------------
# 5. Build X and y, split train/test
# "target_gap_days" (the target column)
# This column is exactly what we are trying to predict:

# The gap between second-last and last purchase (in days).

# We remove it from the feature matrix X, because we never include our target column as an input to the model.

# It goes into y instead:
DROP_COLS = [
    "target_gap_days",        # target
    "email", "Email",         # id columns
    "last_order_date"         # timestamp (not numeric)
]

X = df.drop(columns=DROP_COLS)
y = df["target_gap_days"].astype(float)
# | Split                | Data            | Purpose                           |
# | -------------------- | --------------- | --------------------------------- |
# | `X_train`, `y_train` | 80% of the data | Used to train (fit) the model     |
# | `X_test`, `y_test`   | 20% of the data | Used to test (evaluate) the model |

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42
)

# ------------------------------------------------------------------
# 6. Fit Random-Forest
# ------------------------------------------------------------------
rf = RandomForestRegressor(
    n_estimators=300,
    min_samples_leaf=2,#“Each leaf node (final decision point of any tree) must have at least 2 samples.”
    #Every time  we run the code, we will get exactly the same trained model (same splits, same trees), which is very useful during development.
    random_state=42,
    n_jobs=-1 #“Use all available CPU cores.”

)
rf.fit(X_train, y_train)

# ------------------------------------------------------------------
# 7. Report R²
# ------------------------------------------------------------------
train_r2 = r2_score(y_train, rf.predict(X_train))
test_r2  = r2_score(y_test,  rf.predict(X_test))

print(f"Training R²: {train_r2:.3f}")
print(f"Test (hold-out) R²: {test_r2:.3f}")

# ------------------------------------------------------------------
# 8. (Optional) re-fit on *all* data and generate next-purchase dates
# ------------------------------------------------------------------
rf_all = RandomForestRegressor(
    n_estimators=300,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1
).fit(X, y)

pred_gap = rf_all.predict(X)                 # predicted gap in days
pred_gap_int = np.round(pred_gap).astype(int)

df["predicted_gap_days"] = pred_gap
df["predicted_next_purchase_date"] = (
    df["last_order_date"] + pd.to_timedelta(pred_gap_int, unit="D")
).dt.date

result = df[["email", "predicted_gap_days", "predicted_next_purchase_date"]]
OUT_PRED_PATH = Path("results.csv")   # or an absolute path

result.to_csv(OUT_PRED_PATH, index=False)   # ← suppress the index column
print(f"Saved to: {OUT_PRED_PATH.resolve()}")   # confirms the exact location
import os, pathlib
print("Current working dir:", pathlib.Path().resolve())
print("CSV exists:", OUT_PRED_PATH.exists())


Training R²: 0.734
Test (hold-out) R²: 0.192
Saved to: C:\Users\Public\Inholland\Year 3\Minor\3.4\Moodies Undies\MoodiesData\notebooks\results.csv
Current working dir: C:\Users\Public\Inholland\Year 3\Minor\3.4\Moodies Undies\MoodiesData\notebooks
CSV exists: True
